In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import pdb

In [2]:
df_pickle = '/home/lashi/assets/clean-data/full_data_141516.pickle'
df = pd.read_pickle(df_pickle).reset_index(drop=True)

In [ ]:
# Get time buckets (only needs to be run once)
def createTimeBuckets(slot):
    # loop to create time bucket dictionary
    time_buckets = dict()
    numBuckets = int(24*(60/slot))
    ind = 0
    for hr in range(24):
        for i in range(int(60/slot)):
            key = '%02d'%hr+':'+'%02d'%(i*slot)+':00'
            oneHotVec = np.zeros((1,numBuckets)).astype('int')
            oneHotVec[0,ind] = 1
            time_buckets[key] = ind
            time_buckets[ind] = oneHotVec
            ind += 1

    return time_buckets
    
def parsetime(data_frame, desired_labels, dt_label, timeBuckets):
    data_frame[dt_label] = pd.to_datetime(data_frame[dt_label])
    data_frame[dt_label] = data_frame[dt_label].dt.round('30min')
    data_frame[desired_labels[0]] = data_frame[dt_label].dt.dayofweek
    data_frame[desired_labels[1]] = data_frame[dt_label].dt.time.apply(lambda x: timeBuckets[str(x)])
    data_frame[desired_labels[2]] = data_frame[dt_label].dt.month

tBuckets = createTimeBuckets(30)
desired_labels = ['day_of_week','t_bucket','month']
PU_dt = 'tpep_pickup_datetime'
parsetime(df,desired_labels,PU_dt,tBuckets)
# df.to_pickle(df_pickle)

In [12]:
df_cluster_ids = df.groupby(['pickup_id','cluster_id']).size().reset_index()
df_pickup_ids = df.groupby(['pickup_id']).size().reset_index()
cluster_dist_dict = dict()

for i in range(len(df_pickup_ids)):
    pickup_id = df_pickup_ids['pickup_id'][i]
    id_set = df_cluster_ids.loc[df_cluster_ids['pickup_id']==pickup_id].reset_index()
    num_clusters = len(id_set)
    cluster_dist = np.zeros(num_clusters)
    for cluster in range(num_clusters):
        cluster_dist[cluster] = id_set[0].loc[cluster]
    cluster_dist = cluster_dist/np.sum(cluster_dist)
    cluster_dist_dict[pickup_id] = cluster_dist
    
print(cluster_dist_dict[85])

[0.05319149 0.15957447 0.08510638 0.15957447 0.08510638 0.06382979
 0.08510638 0.07446809 0.20212766 0.03191489]


In [ ]:
print(df2['pickup_id'][255])
print(len(df2))